In [6]:


import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import random
from datetime import datetime

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class InterparkTicketCrawler:
    def __init__(self, creds='google.json', sheet_name='감사한 티켓팅 신청서'):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(creds, scope)
        self.sheet = gspread.authorize(creds).open(sheet_name).worksheet('Hot')

        # 캐시 파일 경로
        self.artist_cache_path = Path('artist_cache.json')
        self.hashtag_cache_path = Path('hashtag_cache.json')
        self.tweet_cache_path = Path('tweet_cache.json')

        # 캐시 로딩
        self.artist_cache = self.load_cache(self.artist_cache_path)
        self.hashtag_cache = self.load_cache(self.hashtag_cache_path)
        self.tweet_cache = self.load_cache(self.tweet_cache_path)
        
    def load_cache(self, path: Path) -> dict:
        if path.exists():
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def save_cache(self, cache: dict, path: Path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)

    def fetch_data(self):
        url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        params = {"goodsGenre": "ALL", "goodsRegion": "ALL", "offset": 0, "pageSize": 400, "sorting": "OPEN_ASC"}
        headers = {
            "user-agent": "Mozilla/5.0",
            "referer": "https://tickets.interpark.com/contents/notice"
        }
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        return r.json()

    def filter_hot(self, data):
        hot = []
        for d in data:
            hot.append({
                '오픈시간': d.get('openDateStr', ''),
                '조회수': d.get('viewCount', 0),
                '예매타입': d.get('openTypeStr', ''),
                '제목': d.get('title', ''),
                '예매코드': d.get('goodsCode', ''),
                '장르': d.get('goodsGenreStr', ''),
                'Image': d.get('posterImageUrl', '')
            })
        return hot


    def run(self):
        raw = self.fetch_data()
        hot = self.filter_hot(raw)
        df = pd.DataFrame(hot)
        if df.empty:
            return df
        df = df.sort_values(by='오픈시간')
        
        return df

df = InterparkTicketCrawler().run()
df.to_csv('all_list.csv', index=False)
if not df.empty:
    print("\n📋 HOT 티켓 요약:")
    print(df[['오픈시간', '조회수','제목',  '장르']].to_string(index=False))


📋 HOT 티켓 요약:
               오픈시간   조회수                                                                    제목      장르
2025-07-17 10:00:00    43                                                2025 라이브가족뮤지컬〈신데렐라〉-대구     뮤지컬
2025-07-17 11:00:00  1574                                                      류이치사카모토 트리뷰트 콘서트 클래식/오페라
2025-07-17 14:00:00   488                                                라라라 크로스오버 페스티벌_A-LiVE  클래식/오페라
2025-07-17 14:00:00   911                                                         뮤지컬 〈스트라빈스키〉      뮤지컬
2025-07-17 14:00:00   309                                             쇼뮤지컬 터치 파이브 〈TOUCH FIVE〉      뮤지컬
2025-07-17 14:00:00  1889                                                             뮤지컬 〈멤피스〉     뮤지컬
2025-07-17 14:00:00   182                                                         파리나무십자가 소년합창단 클래식/오페라
2025-07-17 14:00:00    69                                               카운터테너 이동규 리사이틀 〈바로크로그〉  클래식/오페라
2025-07-17 15:00:00   136                         

In [ ]:

df['장르'].value_counts()

장르
뮤지컬        24
콘서트        18
클래식/오페라     8
연극          6
무용/전통예술     3
Name: count, dtype: int64

In [15]:
df[df['장르']=='콘서트'].sort_values(by='조회수', ascending=False)

,오픈시간,조회수,예매타입,제목,예매코드,장르,Image
57,2025-07-30 20:00:00,37926,일반예매,TOMORROW X TOGETHER WORLD TOUR 〈ACT : TOMORROW...,25008966,콘서트,https://ticketimage.interpark.com/Play/image/l...
11,2025-07-17 18:00:00,6619,일반예매,"너드커넥션 SUMMER LIVE, PULSE",25008903,콘서트,https://ticketimage.interpark.com/Play/image/l...
56,2025-07-25 18:00:00,6611,일반예매,민트페스타 vol.78 SPIRITED,,콘서트,https://ticketimage.interpark.com/TicketImage/...
48,2025-07-23 12:00:00,3737,일반예매,더 로즈(The Rose) Once Upon A WRLD Tour in Seoul,25009724,콘서트,https://ticketimage.interpark.com/Play/image/l...
26,2025-07-18 20:00:00,3122,3차티켓오픈,2025 AKMU STANDING CONCERT ［악동들］,,콘서트,https://ticketimage.interpark.com/TicketImage/...
43,2025-07-22 12:00:00,1248,일반예매,Jacky Cheung 60+ Concert Tour Seoul,,콘서트,https://ticketimage.interpark.com/TicketImage/...
25,2025-07-18 16:00:00,1166,일반예매,"2025 데미소다 콘서트, DEMI-CON!",,콘서트,https://ticketimage.interpark.com/TicketImage/...
49,2025-07-23 16:00:00,974,일반예매,PEAKBOX 2025 : 사랑,,콘서트,https://ticketimage.interpark.com/TicketImage/...
47,2025-07-22 20:00:00,941,일반예매,2025 유채훈 크로스오버 콘서트 〈IL MONDO〉,,콘서트,https://ticketimage.interpark.com/TicketImage/...
51,2025-07-23 18:00:00,798,일반예매,PEAKBOX 2025 : 행복,,콘서트,https://ticketimage.interpark.com/TicketImage/...


In [ ]:


import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import random
from datetime import datetime

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class InterparkTicketCrawler:
    def __init__(self, creds='google.json', sheet_name='감사한 티켓팅 신청서'):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(creds, scope)
        self.sheet = gspread.authorize(creds).open(sheet_name).worksheet('Hot')

        # 캐시 파일 경로
        self.artist_cache_path = Path('artist_cache.json')
        self.hashtag_cache_path = Path('hashtag_cache.json')
        self.tweet_cache_path = Path('tweet_cache.json')

        # 캐시 로딩
        self.artist_cache = self.load_cache(self.artist_cache_path)
        self.hashtag_cache = self.load_cache(self.hashtag_cache_path)
        self.tweet_cache = self.load_cache(self.tweet_cache_path)
        
    def load_cache(self, path: Path) -> dict:
        if path.exists():
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def save_cache(self, cache: dict, path: Path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)

    def fetch_data(self):
        url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        params = {"goodsGenre": "ALL", "goodsRegion": "ALL", "offset": 0, "pageSize": 400, "sorting": "OPEN_ASC"}
        headers = {
            "user-agent": "Mozilla/5.0",
            "referer": "https://tickets.interpark.com/contents/notice"
        }
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        return r.json()
# 뮤지컬, 연극 500 , 클래식/오페라 400, 콘서트 600
    def filter_hot(self, data):
        hot = []
        for d in data:
            if d.get('goodsGenreStr') == '콘서트' and d.get('viewCount', 0) <= 600:
                continue
            if d.get('goodsGenreStr') == '뮤지컬' and d.get('viewCount', 0) <= 500:
                continue
            if d.get('goodsGenreStr') == '연극' and d.get('viewCount', 0) <= 500:
                continue
            if d.get('goodsGenreStr') == '클래식/오페라' and d.get('viewCount', 0) <= 400:
                continue
            
            hot.append({
                '오픈시간': d.get('openDateStr', ''),
                '조회수': d.get('viewCount', 0),
                '예매타입': d.get('openTypeStr', ''),
                '제목': d.get('title', ''),
                '예매코드': d.get('goodsCode', ''),
                '장르': d.get('goodsGenreStr', ''),
                'Image': d.get('posterImageUrl', '')
            })
        return hot

    def extract_artist(self, title: str) -> str:
        if title in self.artist_cache:
            return self.artist_cache[title]

        prompt = f"""
아래는 콘서트 제목이야. 여기서 가수명이나 그룹명만 간단히 추출해줘. 뮤지컬일 경우 뮤지컬 제목만 추출해줘.**영문일 경우 한글도 같이 작성해야되고, 약어가 있으면 풀네임이랑 약어도 같이 작성해야해**
예시: 악동뮤지션 (악뮤, AKMU)
제목: {title}
가수명 or 뮤지컬 제목:"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
            )
            artist = res.choices[0].message.content.strip().strip('"')
            self.artist_cache[title] = artist
            return artist
        except Exception as e:
            print(f"❌ OpenAI 오류 (가수명): {e}")
            return "불명"

    def generate_hashtags(self, title: str, artist: str, genre: str) -> str:
        key = f"{title}"
        if key in self.hashtag_cache:
            return self.hashtag_cache[key]

        prompt = f"""
콘서트 제목: {title}
가수 또는 뮤지컬 제목: {artist}
장르: {genre}

위 콘서트를 대리티켓팅 목적으로 트위터에 해시태그 10개를 한국어로 작성해줘.
형식: #블랙핑크콘서트 #블랙핑크 #BLACKPINK #블핑댈티 #대리티켓팅
조건: '#' 포함하고 띄어쓰기 없이, 한 줄로 콤마 없이 출력해줘.
"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
            )
            hashtags = res.choices[0].message.content.strip()
            self.hashtag_cache[key] = hashtags
            return hashtags
        except Exception as e:
            print(f"❌ OpenAI 오류 (해시태그): {e}")
            return "#대리티켓팅"

    def add_ai_columns(self, df):
        print("🤖 가수명 + 해시태그 생성 중...")
        artists = []
        hashtags = []

        for _, row in tqdm(df.iterrows(), total=len(df)):
            title = row['제목']
            genre = row['장르']

            artist = self.extract_artist(title)
            hashtag = self.generate_hashtags(title, artist, genre)

            artists.append(artist)
            hashtags.append(hashtag)

        df['가수명'] = artists
        df['해시태그'] = hashtags

        self.save_cache(self.artist_cache, self.artist_cache_path)
        self.save_cache(self.hashtag_cache, self.hashtag_cache_path)

        return df
    
    def add_twitter_columns(self, df):
        print("🤖 트위터 문구 생성 중...")
        with open('tweet_templates.json', 'r', encoding='utf-8') as f:
            templates = json.load(f)
        
        tweet_contents = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            

            template = {"content": "{title}\n\n🚨 {singer} 대리티켓팅(댈티)\n\n수고비 제일 저렴\n경력 매우 많음\n\n상담 링크: https://open.kakao.com/o/sAJ8m2Ah\n\n{hash_tag}"}
            
            # if row['조회수'] > 10000:
            #     template = random.choice(templates)
            # else:
            #     template = {"content": "{title}\n\n🚨 {singer} 대리티켓팅(댈티)\n\n수고비 제일 저렴\n경력 매우 많음\n\n상담 링크: https://open.kakao.com/o/sAJ8m2Ah\n\n{hash_tag} #평생한번 #놓치면후회 #앞열보장"}
            
            # 시간 치환
            
            title = row['제목']
            singer = row['가수명']
            
            # 오픈시간이 문자열인 경우 datetime으로 변환
            open_time_raw = row['오픈시간']
            if isinstance(open_time_raw, str):
                # 문자열을 datetime으로 변환
                open_time_dt = datetime.strptime(open_time_raw, '%Y-%m-%d %H:%M:%S')
                open_time = open_time_dt.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            else:
                # 이미 datetime 객체인 경우
                open_time = open_time_raw.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            
            hash_tag = row['해시태그']
            content = template['content'].replace("{open_time}", open_time).replace("{title}", title).replace("{singer}", singer).replace("{hash_tag}", hash_tag)
            tweet_contents.append(content)

        df['트위터'] = tweet_contents
        self.save_cache(self.tweet_cache, self.tweet_cache_path)
        return df
        
    def bunjang_columns(self, df):
        print("🤖 번장 문구 생성 중...")
        
        bunjang_contents = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            

            template = {"content": "{title}\n\n🚨 {singer} 대리티켓팅(댈티)\n\n수고비 제일 저렴\n경력 매우 많음\n\n가격: 번개톡 상담\n\n{hash_tag}"}
            
            
            title = row['제목']
            singer = row['가수명']
            
            # 오픈시간이 문자열인 경우 datetime으로 변환
            open_time_raw = row['오픈시간']
            if isinstance(open_time_raw, str):
                # 문자열을 datetime으로 변환
                open_time_dt = datetime.strptime(open_time_raw, '%Y-%m-%d %H:%M:%S')
                open_time = open_time_dt.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            else:
                # 이미 datetime 객체인 경우
                open_time = open_time_raw.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            
            hash_tag = row['해시태그']
            content = template['content'].replace("{open_time}", open_time).replace("{title}", title).replace("{singer}", singer).replace("{hash_tag}", hash_tag)
            bunjang_contents.append(content)

        df['번장'] = bunjang_contents
        
        return df

    def update_sheet(self, df):
        self.sheet.clear()
        if df.empty:
            print("📭 HOT 티켓 없음")
            return
        self.sheet.append_row(list(df.columns))
        for row in df.values.tolist():
            self.sheet.append_row(row)
        print(f"✅ {len(df)}개 티켓 업로드 완료")

    def run(self):
        raw = self.fetch_data()
        hot = self.filter_hot(raw)
        df = pd.DataFrame(hot)
        if df.empty:
            return df
        df = df.sort_values(by='오픈시간')
        df = self.add_ai_columns(df)
        df = self.add_twitter_columns(df)
        df = self.bunjang_columns(df)
        self.update_sheet(df)
        return df

if __name__ == "__main__":
    df = InterparkTicketCrawler().run()
    if not df.empty:
        print("\n📋 HOT 티켓 요약:")
        print(df[['오픈시간', '제목', '가수명', '해시태그', '번장', '트위터']].to_string(index=False))

🤖 가수명 + 해시태그 생성 중...


100%|██████████| 11/11 [00:15<00:00,  1.42s/it]


🤖 트위터 문구 생성 중...


100%|██████████| 11/11 [00:00<00:00, 10987.70it/s]


🤖 번장 문구 생성 중...


100%|██████████| 11/11 [00:00<00:00, 11003.42it/s]


✅ 11개 티켓 업로드 완료

📋 HOT 티켓 요약:
               오픈시간                                                                    제목                                  가수명                                                                                      해시태그                                                                                                                                                                                                                                        번장                                                                                                                                                                                                                                                                     트위터
2025-07-17 11:00:00                                                      류이치사카모토 트리뷰트 콘서트          류이치 사카모토 (Ryuichi Sakamoto)        #류이치사카모토트리뷰트콘서트 #류이치사카모토 #트리뷰트콘서트 #클래식콘서트 #오페라콘서트 #대리티켓팅 #티켓팅대행 #음악공연 #공연정보 #콘서트소식                                       

In [2]:
import os
import requests
import json
from pathlib import Path
from time import sleep
import random
from tqdm import tqdm

import os
import requests
import json
from pathlib import Path
from time import sleep
import random
from tqdm import tqdm

class PostBunjang:
    def __init__(self, auth_token=None):
        self.auth_token = "53a119a23abe4baa83d75e604dbc2a2d"
        self.location = {
            "address": "서울특별시 서초구 서초4동",
            "lat": 37.5025863,
            "lon": 127.022219,
            "dongId": 648
        }
        os.makedirs("image", exist_ok=True)

    def _download_image(self, url):
        path = f"image/{url.split('/')[-1]}"
        if os.path.exists(path):
            print(f"📁 이미지 이미 존재: {path}")
            return path
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(path, "wb") as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
            print(f"✅ 이미지 다운로드 완료: {path}")
            return path
        print(f"❌ 이미지 다운로드 실패: {url}")
        return None

    def register_bunjang_product(self, image_path, name, description, keywords, price):
        # 1단계: 이미지 업로드
        upload_url = 'https://media-center.bunjang.co.kr/upload/79373298/product'
        upload_headers = {
            'referer': 'https://m.bunjang.co.kr/',
            'user-agent': 'Mozilla/5.0',
            'origin': 'https://m.bunjang.co.kr',
            'accept': 'application/json, text/plain, */*'
        }

        if not Path(image_path).exists():
            print(f"❌ 이미지 파일 없음: {image_path}")
            return None

        with open(image_path, 'rb') as img_file:
            # ✅ 파일명은 항상 ASCII (latin-1 인코딩 문제 방지)
            files = {'file': ('upload.jpg', img_file, 'image/jpeg')}
            upload_res = requests.post(upload_url, headers=upload_headers, files=files)

        if upload_res.status_code != 200:
            print("❌ 이미지 업로드 실패:", upload_res.text)
            return None

        image_id = upload_res.json().get('image_id')
        print("✅ 이미지 업로드 성공:", image_id)

        # 2단계: 상품 등록
        product_url = 'https://api.bunjang.co.kr/api/pms/v2/products'
        product_headers = {
            'content-type': 'application/json',
            'x-bun-auth-token': self.auth_token,
            'user-agent': 'Mozilla/5.0',
            'origin': 'https://m.bunjang.co.kr',
            'referer': 'https://m.bunjang.co.kr/',
            'accept': 'application/json, text/plain, */*'
        }

        # 해시태그 문자열이면 리스트로 변환
        if isinstance(keywords, str):
            keywords = [k.strip() for k in keywords.split('#') if k.strip()]

        product_data = {
            "categoryId": "900210001",
            "common": {
                "description": description,
                "keywords": keywords,
                "name": name,
                "condition": "UNDEFINED",
                "priceOfferEnabled": True
            },
            "option": [],
            "location": {"geo": self.location},
            "transaction": {
                "quantity": 1,
                "price": price,
                "trade": {
                    "freeShipping": True,
                    "isDefaultShippingFee": False,
                    "inPerson": True
                }
            },
            "media": [{"imageId": image_id}],
            "naverShoppingData": {"isEnabled": False}
        }

        res = requests.post(product_url, headers=product_headers, json=product_data)

        if res.status_code == 200:
            pid = res.json().get("data", {}).get("pid", "N/A")
            print("✅ 상품 등록 성공! 🆔", pid)
            return pid
        else:
            print(f"❌ 상품 등록 실패 ({res.status_code}): {res.text}")
            return None

    def post(self, image_url, title, text, hash_tag, price):
        path = self._download_image(image_url)
        if not path:
            return
        pid = self.register_bunjang_product(
            image_path=path,
            name=title,
            description=text,
            keywords=hash_tag,
            price=price
        )
        if pid:
            print(f"🔗 번장 링크: https://m.bunjang.co.kr/products/{pid}")


# ✅ 예시 실행 (df는 미리 정의된 pandas DataFrame이어야 함)
for _, row in tqdm(df.iterrows(), total=len(df)):
    title = row['가수명'] + " 대리티켓팅(댈티)"
    text = row['번장']
    image_url = row['Image']
    tag_str = row['해시태그']
    hash_tag = [tag.strip().lstrip('#') for tag in tag_str.split()][:5]

    price = 9999

    PostBunjang().post(image_url, title, text, hash_tag, price)
    print(f"🔄 {title} 번장 게시 완료")

    sleep_time = random.randint(60, 90)
    for remaining in range(sleep_time, 0, -1):
        print(f"\r⏰ 다음 게시까지 {remaining}초 남음...", end="", flush=True)
        sleep(1)
    print(f"\n⏳ {sleep_time}초 대기 완료")

  0%|          | 0/11 [00:00<?, ?it/s]

✅ 이미지 다운로드 완료: image/25010104_p.gif
✅ 이미지 업로드 성공: 1483496622
❌ 상품 등록 실패 (400): {"errorCode":"ERR_BAD_REQUEST","reason":"키워드를 하나당 9자 이상으로 등록할 수 없습니다."}
🔄 류이치 사카모토 (Ryuichi Sakamoto) 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

  9%|▉         | 1/11 [01:23<13:59, 83.94s/it]


⏳ 83초 대기 완료
✅ 이미지 다운로드 완료: image/25010182_p.gif
✅ 이미지 업로드 성공: 1483497796
❌ 상품 등록 실패 (400): {"errorCode":"ERR_BAD_REQUEST","reason":"키워드를 하나당 9자 이상으로 등록할 수 없습니다."}
🔄 정경화, 케빈 케너 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

 18%|█▊        | 2/11 [02:35<11:30, 76.75s/it]

...


⏳ 82초 대기 완료
📁 이미지 이미 존재: image/2025063015590510.jpg
✅ 이미지 업로드 성공: 1480914949
❌ 상품 등록 실패 (400): {"errorCode":"ERR_BAD_REQUEST","reason":"키워드를 하나당 9자 이상으로 등록할 수 없습니다."}
🔄 민트페스타 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

 91%|█████████ | 10/11 [12:28<01:13, 73.84s/it]


⏳ 60초 대기 완료
📁 이미지 이미 존재: image/25008966_p.gif
✅ 이미지 업로드 성공: 1480915735
❌ 상품 등록 실패 (400): {"errorCode":"ERR_BAD_REQUEST","reason":"상품명을 40자 이하 입력해주세요."}
🔄 TOMORROW X TOGETHER (투모로우바이투게더, TXT) 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

100%|██████████| 11/11 [13:29<00:00, 73.57s/it]


⏳ 60초 대기 완료


In [3]:
import os
import tweepy
import requests
from dotenv import load_dotenv
import time
import random
from tqdm import tqdm

load_dotenv()

class PostTweet:
    def __init__(self):
        auth = tweepy.OAuthHandler(os.getenv('TWITTER_API_KEY'), os.getenv('TWITTER_API_SECRET'))
        auth.set_access_token(os.getenv('TWITTER_ACCESS_TOKEN'), os.getenv('TWITTER_ACCESS_TOKEN_SECRET'))
        self.api = tweepy.API(auth)
        self.client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )

    def _download_image(self, url):
        path = f"image/{url.split('/')[-1]}"
        if os.path.exists(path):
            print(f"이미지 {path} 이미 다운로드됨, 스킵합니다.")
            return path
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(path, "wb") as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
            return path
        return None

    def post(self, text, image_url=None):
        media_ids = []
        if image_url:
            path = self._download_image(image_url)
            if path:
                media = self.api.media_upload(path)
                media_ids.append(media.media_id)

        tweet = self.client.create_tweet(text=text, media_ids=media_ids if media_ids else None)
        print(f"https://twitter.com/gamsahanticket/status/{tweet.data['id']}")


for _, row in tqdm(df.iterrows(), total=len(df)):
    title = row['제목']
    text = row['트위터']
    image_url = row['Image']
    PostTweet().post(text, image_url)
    print(f"🔄 {title} 트윗 게시 완료")
    sleep_time = random.randint(60, 90)
    
    # 실시간 카운트다운
    for remaining in range(sleep_time, 0, -1):
        print(f"\r⏰ 다음 트윗까지 {remaining}초 남음...", end="", flush=True)
        time.sleep(1)
    
    print(f"\n🔄 {sleep_time}초 대기 완료, 다음 트윗 게시")


  0%|          | 0/11 [00:00<?, ?it/s]

이미지 image/25010104_p.gif 이미 다운로드됨, 스킵합니다.
https://twitter.com/gamsahanticket/status/1945422159251378352
🔄 류이치사카모토 트리뷰트 콘서트 트윗 게시 완료
⏰ 다음 트윗까지 1초 남음....

  9%|▉         | 1/11 [01:03<10:30, 63.06s/it]


🔄 61초 대기 완료, 다음 트윗 게시
이미지 image/25010182_p.gif 이미 다운로드됨, 스킵합니다.
https://twitter.com/gamsahanticket/status/1945422422544654836
🔄 정경화 ＆ 케빈 케너 듀오 리사이틀  트윗 게시 완료
⏰ 다음 트윗까지 1초 남음....

 18%|█▊        | 2/11 [02:27<11:22, 75.88s/it]

...


🔄 71초 대기 완료, 다음 트윗 게시
이미지 image/2025063015590510.jpg 이미 다운로드됨, 스킵합니다.
https://twitter.com/gamsahanticket/status/1945425087202836580
🔄 민트페스타 vol.78 SPIRITED 트윗 게시 완료
⏰ 다음 트윗까지 1초 남음....

 91%|█████████ | 10/11 [13:00<01:20, 80.37s/it]


🔄 80초 대기 완료, 다음 트윗 게시
이미지 image/25008966_p.gif 이미 다운로드됨, 스킵합니다.
https://twitter.com/gamsahanticket/status/1945425430976376887
🔄 TOMORROW X TOGETHER WORLD TOUR 〈ACT : TOMORROW〉  IN SEOUL  트윗 게시 완료
⏰ 다음 트윗까지 1초 남음....

100%|██████████| 11/11 [14:09<00:00, 77.19s/it]


🔄 67초 대기 완료, 다음 트윗 게시
